# 필요한 모듈 설치 및 임포트

In [31]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

4.15.2


# 무신사 코디맵 정보 크롤링

In [33]:
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)

codimap_list = []
url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
driver.get(url)
driver.implicitly_wait(2) #2초동안 기다려라
try:
    driver.maximize_window()
except:
    pass

# 페이지를 가장 아래까지 스크롤하는 방법

interval = 1 # 1초에 한번씩 스크롤 내림

# 현재 문서 높이를 가져와서 저장
prev_height = driver.execute_script("return document.body.scrollHeight")

# 반복 수행
while True:
    # 스크롤을 가장 아래로 내림
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

    # 페이지 로딩 대기
    time.sleep(interval)

    # 현재 문서 높이를 가져와서 저장
    curr_height = driver.execute_script("return document.body.scrollHeight")
    if curr_height == prev_height:
        break

    prev_height = curr_height

    # 해당 Xpath 정보를 확인할때까지 5초간 기다림, 5초후에도 해당 Xpath를 찾지 못하면 에러 반환
    # elem = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/ul/li[40]/a/div/img')))

columns = ['codimap_category','codimap_title','codimap_date','views','comment_numbers']      
values = []                                             # 정보를 담을 빈 리스트 생성
soup = BeautifulSoup(driver.page_source, 'lxml')        # 현재 브라우저의 page의 html 정보를 lxml을 이용하여 가져오기
#구문분석기가 달라진다고 해서 크롤링 결과가 크게 달라지진 X
data_rows = soup.find_all('li', attrs={'class':'style-list-item'}) # li tag 중에 class가 style-list-item인 것 list로 반환

for i, row in enumerate(data_rows): # data_rows 리스트에 있는 값들과 인덱스를 차례대로 가져온다
    print('{}번째 코디맵 크롤링'.format(i+1))
    blank = []
    codimap_category = row.find('span', attrs={'class':'style-list-information__text'})   
    if codimap_category:                                              # 해당 조건에 맞는 정보가 하나라도 있을 경우:
        codimap_category = codimap_category.get_text().strip()        # html에서 텍스트만 가져온 뒤 공백 제거
        blank.append(codimap_category)
    else:                                                                       # 해당 조건에 맞는 정보가 하나도 없을 경우:
        blank.append('Something is wrong')
        print('{}번째 codimap_category 가져올때 문제발생'.format(i+1))
        continue
    
    codimap_title = row.find('strong', attrs={'class':'style-list-information__title'})
    if codimap_title:
        codimap_title = codimap_title.get_text().strip()
        blank.append(codimap_title)
    else:
        blank.append('Something is wrong')
        print('{}번째 codimap_title 가져올때 문제발생'.format(i+1))
        continue

        
        
    post_information = row.find_all('span', attrs={'class':'post-information__text'})
    
    codimap_date = post_information[0]
    if codimap_date:
        codimap_date = codimap_date.get_text().strip()
        blank.append(codimap_date)
    else:
        blank.append('Something is wrong')
        print('{}번째 codimap_date 가져올때 문제발생'.format(i+1))
        continue

            
    views = post_information[1] 
    if views:
        views = views.get_text().strip()
        views = views[3:]
        blank.append(views)
    else:
        blank.append('Something is wrong')
        print('{}번째 views 가져올때 문제발생'.format(i+1))
        continue
        
        
    comment_numbers = post_information[2]
    if comment_numbers:
        comment_numbers = comment_numbers.get_text().strip()
        comment_numbers = comment_numbers[3:]
        blank.append(comment_numbers)
    else:
        blank.append('Something is wrong')
        print('{}번째 comment_numbers 가져올때 문제발생'.format(i+1))
        continue
        
    values.append(blank)
    print('---------------------------------------------------')

df = pd.DataFrame(values, columns = columns)
codimap_list.append(df)


1번째 코디맵 크롤링
---------------------------------------------------
2번째 코디맵 크롤링
---------------------------------------------------
3번째 코디맵 크롤링
---------------------------------------------------
4번째 코디맵 크롤링
---------------------------------------------------
5번째 코디맵 크롤링
---------------------------------------------------
6번째 코디맵 크롤링
---------------------------------------------------
7번째 코디맵 크롤링
---------------------------------------------------
8번째 코디맵 크롤링
---------------------------------------------------
9번째 코디맵 크롤링
---------------------------------------------------
10번째 코디맵 크롤링
---------------------------------------------------
11번째 코디맵 크롤링
---------------------------------------------------
12번째 코디맵 크롤링
---------------------------------------------------
13번째 코디맵 크롤링
---------------------------------------------------
14번째 코디맵 크롤링
---------------------------------------------------
15번째 코디맵 크롤링
---------------------------------------------------
16번째 코디맵 크롤링
---------------------

In [34]:
df_ = pd.concat(codimap_list).reset_index(drop=True) # index 초기화 해줘야함
df_.head(10)

,codimap_category,codimap_title,codimap_date,views,comment_numbers
0,캐주얼,톤온톤 코디,21.07.02,"17,239",50
1,캐주얼,블랙 매니아,21.02.23,"36,310",47
2,캐주얼,오늘의 코디,22.03.22,"13,385",44
3,댄디,추천 댄디 룩,21.09.15,"21,583",32
4,캐주얼,릴랙스 룩,21.02.23,"24,015",25
5,스트릿,힙하다 힙해,21.02.03,"36,072",24
6,댄디,데이트 코디,21.12.24,"10,958",19
7,캐주얼,심플한 코디,21.07.30,"41,342",18
8,댄디,훈훈 남친 룩,21.03.24,"19,985",17
9,댄디,서머 데이트 룩,21.07.26,"8,354",17


In [35]:
df_.shape

(60, 5)

# Dataframe을 csv 파일로 저장

In [37]:
df_.to_csv('0105_무신사_코디맵_크롤링.csv', encoding = 'utf-8-sig') # 한글이 들어간 정보는 글자가 깨질 수 있어 encoding을 다음과 같이 지정하여 저장